# PriceTrack: Unlocking Car Market Insights

PriceTrack is a data science project designed to predict the valuation of second-hand cars based on key input parameters. 
Leveraging Linear regression model, it provides data-driven insights to help buyers and sellers make informed decisions.

## Developing Regression Model

Importing all the required modules and functions

In [177]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Loading the cleaned data

In [178]:
df = pd.read_csv("cleaned_data.csv")
df

,name,company,year,Price,kms_driven,fuel_type,Age,Annual_Km_Driven
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,80000,45000,Petrol,13,3461.538462
1,Mahindra Jeep CL550 MDI,Mahindra,2006,425000,140160,Diesel,14,10011.436887
2,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,325000,28000,Petrol,6,4666.666667
3,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,575000,36000,Diesel,6,6000.000000
4,Ford Figo,Ford,2012,175000,41000,Diesel,8,5125.000000
...,...,...,...,...,...,...,...,...
811,Maruti Suzuki Ritz VXI ABS,Maruti,2011,270000,50000,Petrol,9,5555.555556
812,Tata Indica V2 DLE BS III,Tata,2009,110000,30000,Diesel,11,2727.272727
813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol,11,12000.000000
814,Tata Zest XM Diesel,Tata,2018,260000,27000,Diesel,2,13500.000000


### Feature Engineering: Age & Kilometers Driven Transformation  

- **`Age` Calculation:**  
  The car's age is computed as:  
  $$
  \text{Age} = \text{Max Year} - \text{Car's Year} + 1
  $$
  - Newer cars have a **lower age value**, while older cars have a **higher age value** to reflect depreciation.

- **`kms_driven` Inversion:**  
  The **`kms_driven`** column is transformed using:  
  $$
  \text{kms\_driven} = \text{Max kms\_driven} - \text{kms\_driven}
  $$  
  - Higher mileage cars get **lower values** (indicating more wear).  
  - Lower mileage cars get **higher values** (indicating better condition).  

- The original **`year`** column is removed, as it's no longer needed after transformation.  

This transformation improves the model’s ability to capture **price depreciation** over time and with increased usage. 🚗📉  


In [179]:
useful_df = df.copy()
useful_df["Age"] = df["year"].max() - df["year"] + 1  # Assume max year as base year
useful_df = useful_df.drop(columns=["year"])  # Drop original year column
useful_df["kms_driven"] = df["kms_driven"].max() - df["kms_driven"] # Invert kms_driven

# # Replace less frequent companies with "Other"
# top_companies = useful_df["company"].value_counts().nlargest(4).index  # Top 4 companies
# useful_df["company"] = useful_df["company"].apply(lambda x: x if x in top_companies else "Other")

useful_df

,name,company,Price,kms_driven,fuel_type,Age,Annual_Km_Driven
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,80000,155000,Petrol,13,3461.538462
1,Mahindra Jeep CL550 MDI,Mahindra,425000,59840,Diesel,14,10011.436887
2,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,325000,172000,Petrol,6,4666.666667
3,Ford EcoSport Titanium 1.5L TDCi,Ford,575000,164000,Diesel,6,6000.000000
4,Ford Figo,Ford,175000,159000,Diesel,8,5125.000000
...,...,...,...,...,...,...,...
811,Maruti Suzuki Ritz VXI ABS,Maruti,270000,150000,Petrol,9,5555.555556
812,Tata Indica V2 DLE BS III,Tata,110000,170000,Diesel,11,2727.272727
813,Toyota Corolla Altis,Toyota,300000,68000,Petrol,11,12000.000000
814,Tata Zest XM Diesel,Tata,260000,173000,Diesel,2,13500.000000


### Defining Features and Target Variable

- **`X` (Features)**: The independent variables used for prediction:
  - `company`: The brand of the car (categorical).
  - `fuel_type`: The type of fuel the car uses (categorical).
  - `kms_driven`: The total distance the car has been driven (numerical).
  - `Age`: The age of the car, derived from the manufacturing year (numerical).

- **`y` (Target Variable)**:  
  - `Price`: The dependent variable representing the car's selling price.

This selection ensures that the model learns the relationship between these key factors and the car's valuation.


In [180]:
X = useful_df[["company", "fuel_type", "kms_driven", "Age"]]
y = useful_df["Price"]

### Train-Test Split

- The dataset is split into **training** and **testing** sets to evaluate model performance.
- **`train_test_split(X, y, test_size=0.2, random_state=974)`**:
  - **80%** of the data is used for training (`X_train`, `y_train`).
  - **20%** of the data is reserved for testing (`X_test`, `y_test`).
  - `random_state=974` ensures reproducibility by generating the same split every time.

This helps in assessing how well the model generalizes to unseen data.


In [181]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=974)

### Data Preprocessing Pipeline

The `preprocessor` is a `ColumnTransformer` that applies different transformations to categorical and numerical features:

- **One-Hot Encoding (`OneHotEncoder`)**: Converts categorical variables (`company`, `fuel_type`) into numerical format while ignoring unknown categories.
- **Feature Scaling (`StandardScaler`)**: Standardizes numerical features (`kms_driven`, `Age`) to improve model performance by ensuring they have a mean of 0 and a standard deviation of 1.

This preprocessing step ensures that both categorical and numerical data are properly formatted before feeding them into the regression model.


In [182]:
preprocessor = ColumnTransformer([
    ("onehot", OneHotEncoder(handle_unknown='ignore'), ["company", "fuel_type"]),
    ("scaler", StandardScaler(), ["kms_driven", "Age"])
])
preprocessor

ColumnTransformer(transformers=[('onehot',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['company', 'fuel_type']),
                                ('scaler', StandardScaler(),
                                 ['kms_driven', 'Age'])])

### Model Pipeline

A **`Pipeline`** is used to streamline preprocessing and model training in a single workflow:

- **`preprocessor`**: Applies transformations to the input data (One-Hot Encoding & Standard Scaling).
- **`LinearRegression()`**: The regression model that learns the relationship between the features and the target variable.

This approach ensures that preprocessing steps are consistently applied during both training and prediction, improving efficiency and reducing the risk of data leakage.


In [183]:
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

### Training & Predictions

- **`model.fit(X_train, y_train)`**: Trains the Linear Regression model using the training dataset.  
- **`y_pred = model.predict(X_test)`**: Predicts car prices on the test dataset for evaluation.








In [184]:
# Train Model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

### Model Evaluation Metrics

- **Mean Absolute Error (MAE)**: Measures the average absolute difference between actual and predicted prices. Lower values indicate better accuracy.
- **Mean Squared Error (MSE)**: Similar to MAE but gives higher weight to larger errors, making it more sensitive to outliers.
- **Root Mean Squared Error (RMSE)**: Square root of MSE, providing an interpretable error measure in the same unit as price.
- **R² Score**: Indicates how well the model explains the variance in price; closer to 1 means a better fit.

In [185]:
# Model Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Display Metrics
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2 Score: {r2:.2f}")

MAE: 114051.98
MSE: 24190643097.47
RMSE: 155533.41
R2 Score: 0.84


Checking the co-efficients

In [186]:
feature_names = preprocessor.get_feature_names_out()
coef_df = pd.DataFrame(model.named_steps["regressor"].coef_, index=feature_names, columns=["Coefficient"])
coef_df

,Coefficient
onehot__company_Audi,6.598964e+05
onehot__company_BMW,3.490683e+05
onehot__company_Chevrolet,-5.865583e+05
onehot__company_Datsun,-5.986962e+05
onehot__company_Fiat,-5.522360e+05
onehot__company_Force,-3.332595e+05
onehot__company_Ford,-3.096930e+05
onehot__company_Hindustan,-7.464229e+04
onehot__company_Honda,-3.899654e+05
onehot__company_Hyundai,-4.084022e+05


## Integration with Statsmodel

In [187]:
import statsmodels.api as sm

In [188]:
# Encode categorical variables
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)
X_overall_encoded = pd.get_dummies(X, drop_first=True)

# Align the columns of test and overall with train
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
X_overall_encoded = X_overall_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# Add constant (intercept)
X_train_encoded = sm.add_constant(X_train_encoded)
X_test_encoded = sm.add_constant(X_test_encoded)
X_overall_encoded = sm.add_constant(X_overall_encoded)

# Force float dtype
X_train_encoded = X_train_encoded.astype(float)
X_test_encoded = X_test_encoded.astype(float)
X_overall_encoded = X_overall_encoded.astype(float)

y_train = y_train.astype(float)
y_test = y_test.astype(float)
y_overall = y.astype(float)

# Fit the model
model_sm = sm.OLS(y_train, X_train_encoded).fit()

# Predictions
y_train_pred_sm = model_sm.predict(X_train_encoded)
y_test_pred_sm = model_sm.predict(X_test_encoded)
y_overall_pred_sm = model_sm.predict(X_overall_encoded)

results = {
    "r2": [
        model_sm.rsquared,
        r2_score(y_test, y_test_pred_sm),
        r2_score(y_overall, y_overall_pred_sm),
    ],
    "mae": [
        mean_absolute_error(y_train, y_train_pred_sm),
        mean_absolute_error(y_test, y_test_pred_sm),
        mean_absolute_error(y_overall, y_overall_pred_sm),
    ],
    "mse": [
        mean_squared_error(y_train, y_train_pred_sm),
        mean_squared_error(y_test, y_test_pred_sm),
        mean_squared_error(y_overall, y_overall_pred_sm),
    ],
}

results["rmse"] = np.sqrt(results["mse"])
results["r2_percent"] = [f"{val*100:.0f}%" for val in results["r2"]]

# Print Metrics
result_df = pd.DataFrame(
    {
        "Percentage Accuracy": results["r2_percent"],
        "R² Score": results["r2"],
        "MAE": results["mae"],
        "MSE": results["mse"],
        "RMSE": results["rmse"]
    },
    index=["Training", "Testing", "Overall"],
)
# result_df[["R² Score","MAE", "MSE", "RMSE"]].agg(lambda s: ['%.2f'%val for val in s]) # Formatting
result_df

,Percentage Accuracy,R² Score,MAE,MSE,RMSE
Training,66%,0.657335,142021.889519,4.943376e+10,222337.030665
Testing,84%,0.839130,114052.293461,2.419076e+10,155533.806212
Overall,70%,0.695115,136400.549135,4.436041e+10,210619.109079


### Statsmodel Summary on Testing Data

In [189]:
model_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     44.33
Date:                Sun, 27 Apr 2025   Prob (F-statistic):          4.68e-126
Time:                        11:46:14   Log-Likelihood:                -8952.5
No. Observations:                 652   AIC:                         1.796e+04
Df Residuals:                     624   BIC:                         1.809e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               1.754e+06   9.38e+04     18.693      0.000    1.57e+06    1.94e+06
kms_driven            -0.0688      0.322     -0.214      0.831      -0.700       0.563
Age                -3.177e+04   2521.482    -12.598      0.000   -3.67e+04   -2.68e+04
company_BMW        -3.108e+05   1.18e+05     -2.645      0.008   -5.42e+05   -8.01e+04
company_Chevrolet  -1.246e+06   8.58e+04    -14.533      0.000   -1.41e+06   -1.08e+06
company_Datsun     -1.259e+06   1.19e+05    -10.617      0.000   -1.49e+06   -1.03e+06
company_Fiat       -1.212e+06    1.5e+05     -8.076      0.000   -1.51e+06   -9.17e+05
company_Force      -9.932e+05    1.5e+05     -6.624      0.000   -1.29e+06   -6.99e+05
company_Ford       -9.696e+05   8.56e+04    -11.321      0.000   -1.14e+06   -8.01e+05
company_Hindustan  -7.345e+05   1.53e+05     -4.786      0.000   -1.04e+06   -4.33e+05
company_Honda       -1.05e+06      8e+04    -13.128      0.000   -1.21e+06   -8.93e+05
company_Hyundai    -1.068e+06   7.59e+04    -14.081      0.000   -1.22e+06   -9.19e+05
company_Jaguar      6.706e+05   2.39e+05      2.811      0.005    2.02e+05    1.14e+06
company_Jeep       -6.312e+05   2.39e+05     -2.644      0.008    -1.1e+06   -1.62e+05
company_Land         5.17e+05   2.39e+05      2.167      0.031    4.85e+04    9.85e+05
company_Mahindra   -1.002e+06   7.68e+04    -13.060      0.000   -1.15e+06   -8.52e+05
company_Maruti     -1.125e+06   7.45e+04    -15.099      0.000   -1.27e+06   -9.79e+05
company_Mercedes   -1.077e+05   1.35e+05     -0.800      0.424   -3.72e+05    1.57e+05
company_Mini        4.954e+05   1.25e+05      3.957      0.000     2.5e+05    7.41e+05
company_Mitsubishi  -2.45e+05   1.35e+05     -1.820      0.069   -5.09e+05    1.93e+04
company_Nissan     -1.036e+06   1.18e+05     -8.806      0.000   -1.27e+06   -8.05e+05
company_Renault    -1.068e+06   8.43e+04    -12.668      0.000   -1.23e+06   -9.02e+05
company_Skoda      -1.167e+06   9.96e+04    -11.714      0.000   -1.36e+06   -9.71e+05
company_Tata        -1.29e+06   8.02e+04    -16.076      0.000   -1.45e+06   -1.13e+06
company_Toyota     -8.547e+05   8.39e+04    -10.183      0.000   -1.02e+06    -6.9e+05
company_Volkswagen -1.077e+06   9.18e+04    -11.729      0.000   -1.26e+06   -8.97e+05
company_Volvo       2.661e+05   2.39e+05      1.116      0.265   -2.02e+05    7.35e+05
fuel_type_Petrol   -1.226e+05   2.34e+04     -5.238      0.000   -1.69e+05   -7.66e+04
==============================================================================
Omnibus:                      407.376   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10742.475
Skew:                           2.305   Prob(JB):                         0.00
Kurtosis:                      22.343   Cond. No.                     

## Model Testing: Manual Test Cases  

To validate the accuracy and behavior of the trained regression model, we run manual test cases with different car attributes. These test cases help check if the model is making reasonable predictions based on the expected trends.  

| **Company** | **Fuel Type** | **Kilometers Driven** | **Age (Years)** | **Description** | **Expected Outcome** |
|------------|-------------|-----------------|-------------|----------------|----------------|
| Toyota     | Petrol      | 1,000           | 1           | New Car with Low Mileage | A high predicted price, as the car is new with low mileage. |
| Honda      | Diesel      | 200,000         | 10          | Old Car with High Mileage | A significantly lower predicted price, as the car has high mileage and is very old. |
| BMW        | Petrol      | 20,000          | 2           | Luxury Car with Low Mileage | A relatively high predicted price, as luxury cars tend to retain value better. |
| Hyundai    | Petrol      | 50,000          | 5           | Mid-range Car with Moderate Usage | A moderate price, considering the car is neither too old nor too new. |
| Maruti     | Petrol      | 150,000         | 8           | Economy Car with High Mileage | A lower predicted price due to high mileage and moderate age. |

### **Key Trends Expected**  
- **Newer cars with lower mileage** should have a **higher price**.  
- **Older cars with high mileage** should have a **lower price**.  
- **Luxury brands (BMW, Jaguar, Mercedes, etc.)** should have a **higher predicted price** compared to economy brands.  
- **Diesel cars** might have a **slightly higher price** than petrol cars due to fuel efficiency.  

These test cases ensure that the model adheres to logical pricing patterns based on mileage, age, and brand value. 🚗💨


In [190]:
# Define test cases as a list of dictionaries
test_cases = [
    { "company": "Toyota", "fuel_type": "Petrol", "kms_driven": 1000, "Age": 1, "Description": "New Car with Low Mileage",},
    { "company": "Honda", "fuel_type": "Diesel", "kms_driven": 200000, "Age": 10, "Description": "Old Car with High Mileage",},
    { "company": "BMW", "fuel_type": "Petrol", "kms_driven": 20000, "Age": 2, "Description": "Luxury Car with Low Mileage",},
    { "company": "Hyundai", "fuel_type": "Petrol", "kms_driven": 50000, "Age": 5, "Description": "Mid-range Car with Moderate Usage"},
    { "company": "Maruti", "fuel_type": "Petrol", "kms_driven": 150000, "Age": 8, "Description": "Economy Car with High Mileage",}
]

# Convert test cases to DataFrame
test_df = pd.DataFrame(test_cases)

# Predict prices for test cases
predicted_prices = model.predict(test_df.drop(columns=["Description"]))

# Add predicted prices to DataFrame
test_df["Predicted Price (₹)"] = [f"₹{price:,.0f}" for price in predicted_prices]

# Display the results in a structured table
test_df

,company,fuel_type,kms_driven,Age,Description,Predicted Price (₹)
0,Toyota,Petrol,1000,1,New Car with Low Mileage,"₹744,464"
1,Honda,Diesel,200000,10,Old Car with High Mileage,"₹372,288"
2,BMW,Petrol,20000,2,Luxury Car with Low Mileage,"₹1,255,218"
3,Hyundai,Petrol,50000,5,Mid-range Car with Moderate Usage,"₹400,385"
4,Maruti,Petrol,150000,8,Economy Car with High Mileage,"₹241,171"


### Model Testing using Statsmodel

In [191]:
# # Convert test cases to DataFrame
# test_df = pd.DataFrame(test_cases)

# Drop 'Description' column and encode categorical variables in the test data
test_cases_encoded = pd.get_dummies(test_df.drop(columns=["Description"]), drop_first=False)

# Add constant (intercept) to the encoded test data
test_cases_encoded = sm.add_constant(test_cases_encoded)

# Ensure the test data has the same columns as the training data (X_overall_encoded)
test_cases_encoded = test_cases_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# Predict prices for test cases using the overall model
predicted_prices = model_sm.predict(test_cases_encoded)

# Add predicted prices to the test DataFrame
test_df["Predicted Price (₹)"] = [f"₹{price:,.0f}" for price in predicted_prices]

# Display the results in a structured table
test_df


,company,fuel_type,kms_driven,Age,Description,Predicted Price (₹)
0,Toyota,Petrol,1000,1,New Car with Low Mileage,"₹744,464"
1,Honda,Diesel,200000,10,Old Car with High Mileage,"₹372,287"
2,BMW,Petrol,20000,2,Luxury Car with Low Mileage,"₹1,255,213"
3,Hyundai,Petrol,50000,5,Mid-range Car with Moderate Usage,"₹400,386"
4,Maruti,Petrol,150000,8,Economy Car with High Mileage,"₹241,171"
